<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [23]:
import requests
import pandas as pd

In [24]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: ' '.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [25]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,"AS YOU LIKE IT DRAMATIS PERSONAE. DUKE, l..."
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS Contents ACT I Scene ...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS Dramatis Personae ...
4,CYMBELINE,30347,30364,CYMBELINE. Laud we the gods; And let our crook...


In [26]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [27]:
len(df_toc['text'])

43

In [28]:
# Encode df_toc['text'] as Chars

# Gather all text 
# Why? 1. See all possible characters 2. For training / splitting later
text = " ".join(df_toc['text'][10:20])

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [29]:
char_int

{'\t': 69,
 ' ': 3,
 '!': 19,
 '"': 14,
 '%': 68,
 '&': 34,
 "'": 0,
 '(': 47,
 ')': 75,
 '*': 37,
 ',': 88,
 '-': 24,
 '.': 89,
 '/': 93,
 '0': 43,
 '1': 23,
 '2': 11,
 '3': 74,
 '4': 78,
 '5': 18,
 '6': 36,
 '7': 81,
 '8': 16,
 '9': 49,
 ':': 7,
 ';': 70,
 '?': 12,
 'A': 61,
 'B': 40,
 'C': 38,
 'D': 53,
 'E': 65,
 'F': 77,
 'G': 58,
 'H': 55,
 'I': 9,
 'J': 29,
 'K': 90,
 'L': 26,
 'M': 2,
 'N': 52,
 'O': 39,
 'P': 25,
 'Q': 57,
 'R': 51,
 'S': 79,
 'T': 64,
 'U': 5,
 'V': 33,
 'W': 22,
 'X': 73,
 'Y': 13,
 'Z': 62,
 '[': 59,
 '\\': 67,
 ']': 82,
 '_': 86,
 '`': 56,
 'a': 21,
 'b': 20,
 'c': 60,
 'd': 31,
 'e': 66,
 'f': 94,
 'g': 71,
 'h': 46,
 'i': 84,
 'j': 85,
 'k': 45,
 'l': 15,
 'm': 27,
 'n': 1,
 'o': 91,
 'p': 63,
 'q': 6,
 'r': 54,
 's': 17,
 't': 50,
 'u': 4,
 'v': 32,
 'w': 72,
 'x': 8,
 'y': 44,
 'z': 80,
 '|': 48,
 '}': 35,
 'Æ': 92,
 'æ': 41,
 'è': 10,
 'œ': 28,
 '—': 87,
 '‘': 76,
 '’': 30,
 '“': 83,
 '”': 42}

In [30]:
int_char[75]

')'

In [31]:
len(chars)

95

In [32]:
# Create the sequence data

maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  816495


In [33]:
len(text)

4082511

In [34]:
sequences[0]

[64,
 55,
 65,
 3,
 79,
 65,
 38,
 39,
 52,
 53,
 3,
 25,
 61,
 51,
 64,
 3,
 39,
 77,
 3,
 90,
 9,
 52,
 58,
 3,
 55,
 65,
 52,
 51,
 13,
 3,
 64,
 55,
 65,
 3,
 79,
 9,
 73,
 64,
 55,
 3]

In [35]:
# Create x & y

# Padding!
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1
        

In [36]:
x.shape

(816495, 40, 95)

In [37]:
y.shape

(816495, 95)

In [38]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               114688    
_________________________________________________________________
dense_1 (Dense)              (None, 95)                12255     
Total params: 126,943
Trainable params: 126,943
Non-trainable params: 0
_________________________________________________________________


In [40]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [41]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    # Random prompt
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        # Predict the next step (character)
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [42]:
# fit the model

model.fit(x, y,
          batch_size=512,
          epochs=1,
          callbacks=[print_callback])

1590/1595 [============================>.] - ETA: 0s - loss: 2.5192
----- Generating text after Epoch: 0
----- Generating with seed: "   To quench mine honour; they would sha"
   To quench mine honour; they would shasfof.  TAPRIEUCOW. Snoo t ly, arlt'der,;           und afangunger owCorf, Jues ald a? ar Arpseakentse thenrel you these and, co’cfosse whand rome, y tot, Mraull tais foat he iprot o Thome berz. Axurotef thak, youn; Budiont Thath to sheld bobe sared o’d airin, pesen tiep Jon.       And erperan’d I dound ill.    THTaNCEFSHIFWIU.  nndees.  irrfrien?  IWERCHE. And lost  he iel sinove, hivee to hanste.
1595/1595 [==============================] - 31s 20ms/step - loss: 2.5182


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN